## 📌 Descripción del Problema

Resolver un **Sudoku clásico** consiste en llenar una cuadrícula de \(9 \times 9\) con números del 1 al 9, cumpliendo con las siguientes reglas:

- Cada fila debe contener exactamente una vez cada número del 1 al 9.
- Cada columna debe contener exactamente una vez cada número del 1 al 9.
- Cada una de las 9 subcuadrículas \(3 \times 3\) (subcuadros) también debe contener exactamente una vez cada número del 1 al 9.
- Algunas celdas tienen valores predefinidos (datos del tablero inicial).

El objetivo es encontrar una asignación de valores a las celdas que cumpla con todas las restricciones anteriores.

---

## 🧮 Modelo Matemático

### Variables

$$
y_{r,c,v} =
\begin{cases}
1 & \text{si se asigna el valor } v \text{ a la celda } (r,c) \\
0 & \text{en otro caso}
\end{cases}
$$

donde \( r, c, v \in \{1, 2, ..., 9\} \)

---

### Función Objetivo

Este es un problema de **factibilidad**, por lo tanto, la función objetivo es arbitraria (puede ser constante):

$$
\min 1
$$

---

### Restricciones

#### 1. Cada fila debe contener exactamente un valor \(v\):

$$
\sum_{c=1}^{9} y_{r,c,v} = 1 \quad \forall r \in \{1,\dots,9\},\ \forall v \in \{1,\dots,9\}
$$

#### 2. Cada columna debe contener exactamente un valor \(v\):

$$
\sum_{r=1}^{9} y_{r,c,v} = 1 \quad \forall c \in \{1,\dots,9\},\ \forall v \in \{1,\dots,9\}
$$

#### 3. Cada subcuadro \(3 \times 3\) debe contener exactamente un valor \(v\):

Para cada subcuadro \(s \in \{1,\dots,9\}\), definido por un conjunto de coordenadas \((r,c)\in S_s\):

$$
\sum_{(r,c) \in S_s} y_{r,c,v} = 1 \quad \forall v \in \{1,\dots,9\}
$$

#### 4. Cada celda debe tener exactamente un valor:

$$
\sum_{v=1}^{9} y_{r,c,v} = 1 \quad \forall r, c \in \{1,\dots,9\}
$$

#### 5. Valores dados del tablero (si los hay):

Para cada celda predefinida con valor \(v\) en la posición \((r,c)\):

$$
y_{r,c,v} = 1
$$


In [1]:
import pyomo.environ as pyo
from pyomo.core.expr.current import LinearExpression

# Definimos el mapeo de subcuadrantes a celdas
subsq_to_row_col = {
    1: [(i,j) for i in range(1,4) for j in range(1,4)],
    2: [(i,j) for i in range(1,4) for j in range(4,7)],
    3: [(i,j) for i in range(1,4) for j in range(7,10)],
    4: [(i,j) for i in range(4,7) for j in range(1,4)],
    5: [(i,j) for i in range(4,7) for j in range(4,7)],
    6: [(i,j) for i in range(4,7) for j in range(7,10)],
    7: [(i,j) for i in range(7,10) for j in range(1,4)],
    8: [(i,j) for i in range(7,10) for j in range(4,7)],
    9: [(i,j) for i in range(7,10) for j in range(7,10)]
}

def create_sudoku_model(board):
    model = pyo.ConcreteModel()

    model.ROWS = pyo.RangeSet(1,9)
    model.COLS = pyo.RangeSet(1,9)
    model.VALUES = pyo.RangeSet(1,9)
    model.SUBSQUARES = pyo.RangeSet(1,9)

    model.y = pyo.Var(model.ROWS, model.COLS, model.VALUES, within=pyo.Binary)

    # Fijar valores dados
    for (r, c, v) in board:
        model.y[r, c, v].fix(1)

    # Función objetivo trivial (problema de factibilidad)
    model.obj = pyo.Objective(expr=1.0)

    # Restricción: una única aparición de cada valor en cada fila
    def row_rule(model, r, v):
        return LinearExpression(
            constant=0,
            linear_coefs=[1]*9,
            linear_vars=[model.y[r, c, v] for c in model.COLS]
        ) == 1
    model.RowCon = pyo.Constraint(model.ROWS, model.VALUES, rule=row_rule)

    # Restricción: una única aparición de cada valor en cada columna
    def col_rule(model, c, v):
        return LinearExpression(
            constant=0,
            linear_coefs=[1]*9,
            linear_vars=[model.y[r, c, v] for r in model.ROWS]
        ) == 1
    model.ColCon = pyo.Constraint(model.COLS, model.VALUES, rule=col_rule)

    # Restricción: una única aparición de cada valor en cada subcuadro
    def subsquare_rule(model, s, v):
        coords = subsq_to_row_col[s]
        return LinearExpression(
            constant=0,
            linear_coefs=[1]*9,
            linear_vars=[model.y[r, c, v] for (r, c) in coords]
        ) == 1
    model.SquareCon = pyo.Constraint(model.SUBSQUARES, model.VALUES, rule=subsquare_rule)

    # Restricción: cada celda debe tener un único valor
    def cell_rule(model, r, c):
        return LinearExpression(
            constant=0,
            linear_coefs=[1]*9,
            linear_vars=[model.y[r, c, v] for v in model.VALUES]
        ) == 1
    model.CellCon = pyo.Constraint(model.ROWS, model.COLS, rule=cell_rule)

    return model


expression symbols from pyomo.core.expr  (deprecated in 6.6.2) (called from
C:\Users\a_cor\AppData\Local\Temp\ipykernel_15348\120685233.py:2)


In [2]:
from pyomo.opt import SolverFactory, TerminationCondition

# Tablero inicial (como en el libro)
board = [
    (1,1,5),(1,2,3),(1,5,7),
    (2,1,6),(2,4,1),(2,5,9),(2,6,5),
    (3,2,9),(3,3,8),(3,8,6),
    (4,1,8),(4,5,6),(4,9,3),
    (5,1,4),(5,4,8),(5,6,3),(5,9,1),
    (6,1,7),(6,5,2),(6,9,6),
    (7,2,6),(7,7,2),(7,8,8),
    (8,4,4),(8,5,1),(8,6,9),(8,9,5),
    (9,5,8),(9,8,7),(9,9,9)
]

model = create_sudoku_model(board)

solver = SolverFactory('glpk')
result = solver.solve(model)

# Mostrar solución
def print_solution(model):
    for r in model.ROWS:
        fila = ''
        for c in model.COLS:
            for v in model.VALUES:
                if pyo.value(model.y[r, c, v]) >= 0.5:
                    fila += f'{v} '
        print(fila)

if result.solver.termination_condition == TerminationCondition.optimal:
    print("✅ Sudoku resuelto:")
    print_solution(model)
else:
    print("❌ No se encontró solución.")


✅ Sudoku resuelto:
5 3 4 6 7 8 9 1 2 
6 7 2 1 9 5 3 4 8 
1 9 8 3 4 2 5 6 7 
8 5 9 7 6 1 4 2 3 
4 2 6 8 5 3 7 9 1 
7 1 3 9 2 4 8 5 6 
9 6 1 5 3 7 2 8 4 
2 8 7 4 1 9 6 3 5 
3 4 5 2 8 6 1 7 9 
